# **M3_AI3_PREPARAR DATOS DE FACEBOOK POR PAÍS**

---





- Peredo Mescco, Elizabeth Evelin

El objetivo de esta actividad consiste en realizar la preparación de datos inicial para analizar, en fases posteriores, las métricas y mejorar la calidad de los datos.

Los datos a preparar están contenidos en la tabla FACEBOOK_2010_2018_RAW.csv.

- Pais: nombre del país. Un mismo país puede variar su nombre según el indicador.
- Valor: Valores numéricos. Puede presentar formatos distintos, según el indicador.
- Indicador: Formato: Año | indicador:
  - Año: 2010 o 2018.
  - Indicador:
    - Facebook: número de usuarios de Facebook en todo el país.
    - Internet_por: porcentaje de población usuaria de internet.
    - poblacion: total de población por país.

In [ ]:
R.version.string
print(installed.packages())

In [ ]:
library(readr) 
library(tidyr)
library(stringr)
library(dplyr)
library(ggplot2)

In [ ]:
getwd()
download.file("https://raw.githubusercontent.com/griu/mbdds_fc20/master/gestion_datos/www/Facebook_10_18_raw.csv", "/content/dnld_direct_from_web.csv")

# 1. Carga los datos del fichero en un data frame. ¿Por qué no es un conjunto de datos ordenado?


In [ ]:
 df <- read.csv("/content/dnld_direct_from_web.csv", sep=";", header=TRUE)
head(df)

# 2. Separa el año y el “indicador” en dos columnas separadas: anyo, variable. Presenta la tabla de las frecuencias de cada nueva variable.


In [ ]:
nombres_columnas <- colnames(df)
nombres_columnas

In [ ]:
df_new <- cbind(df)
df_new <- separate(df, col = 'indicador', into = c('anyo', 'variable'), sep = '\\|')

table(df_new$anyo)
table(df_new$variable)

head(df_new)

# 3. Codifica el nombre de los países en una nueva columna “pais_r” donde, al analizar la tabla de frecuencias cada país, tenga frecuencia 6.


In [ ]:
df_filter6 <- df_new %>%
  group_by(pais) %>%
  summarise(pais_n = n()) %>%
  filter(pais_n != 6)

df_pais <- df_new

# Cambiar el orden de las palabras
df_pais$pais_r <- gsub(pattern = "^(.*), (.*)$", replacement = "\\2 \\1", x = df_pais$pais)

df_pais <- mutate(df_pais,
                  pais_r = case_when(#pais_r == "Egypt, Arab Rep." ~ "Arab Rep. Egypt",
                                     #pais_r == "Yemen, Rep." ~ "Rep. Yemen",
                                     pais_r == "Russian Federation" ~ "Russian",
                                     TRUE ~ pais_r))

head(df_filter6)
head(df_pais)

# 4. Pivota la tabla a formato wide de forma que cada variable tenga su propia columna y cada país dos filas (una para cada año). Comenta la estructura de la tabla (glimpse).


In [ ]:
df_var <- df_pais[,c('pais_r','anyo','valor','variable')] %>%
  pivot_wider(names_from = c(variable), values_from = c(valor))

glimpse(df_var)

head(df_var)

La función glimpse() muestra un resumen de df_var:


*   **pais_r**: un vector de caracteres que representa los nombres de los países.
*   **anyo**: un vector de caracteres que representa el año de la observación.
*   **Internet_por**: un vector de caracteres que representa el porcentaje de personas en el país que usan internet.
*   **Facebook**: un vector de caracteres que representa el número de usuarios de Facebook en el país.
*   **poblacion**: un vector de caracteres que representa la población del país.

Todas las columnas del dataframe son vectores de caracteres, lo que significa que es necesario convertir algunos de ellos a tipos de datos numéricos para realizar cualquier análisis. 

# 5. Transforma las tres variables (Facebook, Internet_por, poblacion) a formato numérico en tres variables nuevas con el sufijo “_num”. Aplica antes la/s transformación/es de las variables de texto que creas conveniente. Presenta el summary de las variables nuevas.


In [ ]:
df_var$Internet_por_num <- ifelse(is.na(df_var$Internet_por), 0, round(as.numeric(str_replace_all(df_var$Internet_por, "\\s|,", ".")), 3))
df_var$Facebook_num <- ifelse(is.na(df_var$Facebook), 0, as.numeric(str_replace_all(df_var$Facebook, "\\s|,", ".")))
df_var$poblacion_num <- as.numeric(gsub(",", ".", gsub("\\.", "", df_var$poblacion)))

head(df_var)

In [ ]:
summary(df_var)

# 6. Analiza y comenta si observas alguna otra inconsistencia en los datos preparados.


In [ ]:
df1 <- df_var
df1$Internet_num <-round(as.numeric(df1$poblacion_num)*as.numeric(df1$Internet_por_num)/100)
df1$Facebook_por_Internet <- round(as.numeric(df1$Facebook_num)*100/as.numeric(df1$Internet_num),2)

summary(df1)

Existe una inconsistencia entre los datos de la cantidad total de usuarios que utilizan internet y los usuarios que usan Facebook, en este caso en específico que se creó una nueva variable Facebook_por_Internet que debería estar en un rango de 0 a 1, debido a que dentro de los usuarios que usan internet están aquellos que usan Facebook, sin embargo, el valor máximo de este último es mayor al del uso de internet.

# 7. Escribe cuatro líneas explicando (para un lector no técnico) la penetración de Facebook y de Internet en España.


In [ ]:
df_spain <- df1

df_spain <- df_spain[,c('pais_r','anyo','poblacion_num','Internet_num','Internet_por_num','Facebook_num','Facebook_por_Internet')]
df_spain <- df_spain[df_spain$pais_r == 'Spain', ]
df_spain <- df_spain[order(df_spain$pais_r, decreasing = TRUE), ]

df_spain

Según la tabla precedente, la población de España en 2018 era de 46723 de personas, y de ese total, 40232 personas utilizaban Internet, lo que representa una penetración del 86,1% en la población. Esto significa que la gran mayoría de las personas en España tienen acceso a la red.

En cuanto a la penetración de Facebook, en 2010, 11 millones de personas usaban esta red social, en 2018 había 23 millones de personas en España, lo que indica que el uso de esta red social se ha multiplicado significativamente en los últimos años.

En resumen, los datos presentados sugieren que tanto la penetración de Internet como la de Facebook han aumentado significativamente en España en los últimos años. Esto indica que cada vez más personas tienen acceso a la red y están utilizando las redes sociales en su vida cotidiana.